In [77]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [78]:
x_train = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/Voltage_L1_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/output_train.csv")
x_test = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/Voltage_L1_test.csv")
y_test = pd.read_csv("/content/drive/MyDrive/DataForCheckpoint2/DataForCheckpoint2/output_test.csv")

In [79]:
print("x_train",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

x_train (5999, 256)
y_train (5999, 1)
x_test (3599, 256)
y_test (3599, 1)


In [80]:
header =[]
for i in range(1,x_train.shape[1]+1):
    header.append("Col"+str(i))
  
x_train.columns = header
x_test.columns = header

In [81]:
header = ["output"]
y_train.columns = header
y_test.columns = header

In [82]:
x_train.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,...,Col217,Col218,Col219,Col220,Col221,Col222,Col223,Col224,Col225,Col226,Col227,Col228,Col229,Col230,Col231,Col232,Col233,Col234,Col235,Col236,Col237,Col238,Col239,Col240,Col241,Col242,Col243,Col244,Col245,Col246,Col247,Col248,Col249,Col250,Col251,Col252,Col253,Col254,Col255,Col256
0,573.652486,1003.343736,1588.404525,2317.576741,2804.364311,3225.322510,3662.821690,4174.627969,4656.244143,4939.070130,5206.554969,5592.545618,5696.514087,5880.479030,6044.201767,6164.119454,6045.777289,5942.331357,5803.334028,5705.353664,5492.898339,5233.199948,4817.947481,4389.685974,4109.868347,3655.578671,3150.155051,2488.782970,2105.222495,1470.890003,827.651237,333.571731,-216.228004,-859.757556,-1425.521699,-2003.980150,-2570.318330,-3146.343656,-3557.285065,-4000.158443,...,4146.318576,3598.860474,3066.713426,2656.610110,2104.886643,1419.066841,828.590183,268.878130,-261.387355,-874.110382,-1543.916944,-2105.454411,-2543.234642,-2985.412538,-3568.535598,-3951.801096,-4374.196247,-4754.425027,-5191.577616,-5299.392099,-5628.953870,-5796.512723,-5751.654477,-5961.074990,-5983.366440,-5787.743153,-5760.805708,-5565.520345,-5231.293062,-5026.769575,-4650.282434,-4228.581226,-3865.609932,-3395.654756,-2933.680470,-2322.450904,-1841.562453,-1282.042025,-601.968217,-156.848367
1,4757.365183,5264.598912,5428.642486,5650.413073,5939.710012,5911.948067,6147.642171,6076.921501,5958.797444,6053.817701,5728.841911,5644.998149,5298.732396,4974.047818,4591.611691,4186.262967,3733.027881,3255.865670,2830.180005,2175.293493,1727.613089,1187.819709,558.834809,-143.577582,-569.246545,-1267.750839,-1739.441049,-2316.246419,-2920.988706,-3434.114162,-3741.577681,-4234.018706,-4526.556346,-5013.305861,-5183.721752,-5581.875945,-5746.412108,-5878.049484,-5787.672448,-5883.287834,...,-650.549022,-1196.759735,-1774.293292,-2410.101811,-2911.547024,-3345.463756,-3910.845533,-4262.833689,-4669.800881,-4874.912445,-5233.309917,-5426.858323,-5683.336421,-5760.747433,-5963.086618,-5810.188138,-5880.373421,-5811.043872,-5612.603743,-5444.308315,-5180.722869,-4879.118026,-4453.158606,-4019.414925,-3562.978086,-3144.555305,-2532.911641,-2023.992358,-1489.870210,-884.986837,-280.360872,323.325836,861.103019,1415.929276,2007.692919,2561.130303,2960.282598,3619.932691,4008.288701,4422.229911
2,4242.144824,4644.679402,5013.356532,5229.417051,5534.898007,5797.190678,5930.658682,5960.014599,6055.336310,6103.707793,6004.988201,5808.771978,5606.283312,5468.436913,5069.759820,4834.935692,4396.020933,3943.531449,3513.035844,2988.029609,2450.632217,1869.727370,1417.929664,759.192139,145.610588,-391.912785,-1040.230423,-1574.897173,-2090.768570,-2594.147511,-3152.815772,-3611.369855,-4134.024439,-4535.638814,-4879.053974,-5191.111517,-5415.211621,-5620.717615,-5838.524339,-5865.683887,...,163.182447,-393.247934,-961.630617,-1578.787165,-2153.359595,-2608.889538,-3107.800507,-3601.042587,-4052.041291,-4435.032183,-4903.126311,-5187.215581,-5409.919307,-5586.551795,-5800.681672,-5932.394619,-5969.248062,-5916.716780,-5835.630240,-5664.542367,-5532.906589,-5267.144730,-5057.089356,-4692.472076,-4294.789780,-3829.670275,-3404.173134,-2841.728040,-2345.290048,-1804.647738,-1256.270585,-616.527428,-67.068193,549.016676,1099.652199,1697.572166,2239.961604,2776.876479,3248.638662,3807.665149
3,2077.819247,2561.679246,3085.653813,3545.905160,4023.421592,4496.705157,4809.079868,5186.298840,5453.627533,5737.354699,5810.054347,6004.284697,5994.962436,6099.595124,5969.767022,5876.561359,5732.920266,5496.224936,5211.231518,4864.708340,4529.038028,4100.012484,3708.373312,3178.175298,2654.625182,2079.096940,1589.340001,989.824249,424.103242,-166.638772,-766.543652,-1318.951263,-1934.174920,-2491.291919,-2985.793735,-3535.006312,-3907.884009,-4308.751786,-4779.143197,-504

In [83]:
x_train

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,...,Col217,Col218,Col219,Col220,Col221,Col222,Col223,Col224,Col225,Col226,Col227,Col228,Col229,Col230,Col231,Col232,Col233,Col234,Col235,Col236,Col237,Col238,Col239,Col240,Col241,Col242,Col243,Col244,Col245,Col246,Col247,Col248,Col249,Col250,Col251,Col252,Col253,Col254,Col255,Col256
0,573.652486,1003.343736,1588.404525,2317.576741,2804.364311,3225.322510,3662.821690,4174.627969,4656.244143,4939.070130,5206.554969,5592.545618,5696.514087,5880.479030,6044.201767,6164.119454,6045.777289,5942.331357,5803.334028,5705.353664,5492.898339,5233.199948,4817.947481,4389.685974,4109.868347,3655.578671,3150.155051,2488.782970,2105.222495,1470.890003,827.651237,333.571731,-216.228004,-859.757556,-1425.521699,-2003.980150,-2570.318330,-3146.343656,-3557.285065,-4000.158443,...,4146.318576,3598.860474,3066.713426,2656.610110,2104.886643,1419.066841,828.590183,268.878130,-261.387355,-874.110382,-1543.916944,-2105.454411,-2543.234642,-2985.412538,-3568.535598,-3951.801096,-4374.196247,-4754.425027,-5191.577616,-5299.392099,-5628.953870,-5796.512723,-5751.654477,-5961.074990,-5983.366440,-5787.743153,-5760.805708,-5565.520345,-5231.293062,-5026.769575,-4650.282434,-4228.581226,-3865.609932,-3395.654756,-2933.680470,-2322.450904,-1841.562453,-1282.042025,-601.968217,-156.848367
1,4757.365183,5264.598912,5428.642486,5650.413073,5939.710012,5911.948067,6147.642171,6076.921501,5958.797444,6053.817701,5728.841911,5644.998149,5298.732396,4974.047818,4591.611691,4186.262967,3733.027881,3255.865670,2830.180005,2175.293493,1727.613089,1187.819709,558.834809,-143.577582,-569.246545,-1267.750839,-1739.441049,-2316.246419,-2920.988706,-3434.114162,-3741.577681,-4234.018706,-4526.556346,-5013.305861,-5183.721752,-5581.875945,-5746.412108,-5878.049484,-5787.672448,-5883.287834,...,-650.549022,-1196.759735,-1774.293292,-2410.101811,-2911.547024,-3345.463756,-3910.845533,-4262.833689,-4669.800881,-4874.912445,-5233.309917,-5426.858323,-5683.336421,-5760.747433,-5963.086618,-5810.188138,-5880.373421,-5811.043872,-5612.603743,-5444.308315,-5180.722869,-4879.118026,-4453.158606,-4019.414925,-3562.978086,-3144.555305,-2532.911641,-2023.992358,-1489.870210,-884.986837,-280.360872,323.325836,861.103019,1415.929276,2007.692919,2561.130303,2960.282598,3619.932691,4008.288701,4422.229911
2,4242.144824,4644.679402,5013.356532,5229.417051,5534.898007,5797.190678,5930.658682,5960.014599,6055.336310,6103.707793,6004.988201,5808.771978,5606.283312,5468.436913,5069.759820,4834.935692,4396.020933,3943.531449,3513.035844,2988.029609,2450.632217,1869.727370,1417.929664,759.192139,145.610588,-391.912785,-1040.230423,-1574.897173,-2090.768570,-2594.147511,-3152.815772,-3611.369855,-4134.024439,-4535.638814,-4879.053974,-5191.111517,-5415.211621,-5620.717615,-5838.524339,-5865.683887,...,163.182447,-393.247934,-961.630617,-1578.787165,-2153.359595,-2608.889538,-3107.800507,-3601.042587,-4052.041291,-4435.032183,-4903.126311,-5187.215581,-5409.919307,-5586.551795,-5800.681672,-5932.394619,-5969.248062,-5916.716780,-5835.630240,-5664.542367,-5532.906589,-5267.144730,-5057.089356,-4692.472076,-4294.789780,-3829.670275,-3404.173134,-2841.728040,-2345.290048,-1804.647738,-1256.270585,-616.527428,-67.068193,549.016676,1099.652199,1697.572166,2239.961604,2776.876479,3248.638662,3807.665149
3,2077.819247,2561.679246,3085.653813,3545.905160,4023.421592,4496.705157,4809.079868,5186.298840,5453.627533,5737.354699,5810.054347,6004.284697,5994.962436,6099.595124,5969.767022,5876.561359,5732.920266,5496.224936,5211.231518,4864.708340,4529.038028,4100.012484,3708.373312,3178.175298,2654.625182,2079.096940,1589.340001,989.824249,424.103242,-166.638772,-766.543652,-1318.951263,-1934.174920,-2491.291919,-2985.793735,-3535.006312,-3907.884009,-4308.751786,-4779.143197,-504

In [84]:
x_train = np.asarray(x_train).reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = np.asarray(x_test).reshape(x_test.shape[0],x_test.shape[1], 1)

In [99]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv1D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(256,1)
    ),
    keras.layers.Conv1D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(7, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [100]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [102]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='content',project_name="Classification")

In [103]:
tuner_search.search(x_train,y_train,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 13s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 48s
INFO:tensorflow:Oracle triggered exit


In [104]:
model=tuner_search.get_best_models(num_models=1)[0]

In [105]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 252, 48)           288       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 250, 64)           9280      
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1024064   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 455       
Total params: 1,034,087
Trainable params: 1,034,087
Non-trainable params: 0
_________________________________________________________________


In [106]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
169/169 [==============================] - 5s 26ms/step - loss: 0.2178 - accuracy: 0.9418 - val_loss: 0.0072 - val_accuracy: 0.9983
Epoch 5/10
169/169 [==============================] - 4s 25ms/step - loss: 0.0397 - accuracy: 0.9892 - val_loss: 1.3633e-04 - val_accuracy: 1.0000
Epoch 6/10
169/169 [==============================] - 4s 25ms/step - loss: 0.0183 - accuracy: 0.9945 - val_loss: 0.0103 - val_accuracy: 0.9967
Epoch 7/10
169/169 [==============================] - 4s 25ms/step - loss: 0.0336 - accuracy: 0.9926 - val_loss: 0.0247 - val_accuracy: 0.9867
Epoch 8/10
169/169 [==============================] - 4s 25ms/step - loss: 0.1169 - accuracy: 0.9620 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 9/10
169/169 [==============================] - 4s 25ms/step - loss: 0.0357 - accuracy: 0.9877 - val_loss: 0.0088 - val_accuracy: 0.9983
Epoch 10/10
169/169 [==============================] - 4s 25ms/step - loss: 0.0097 - accuracy: 0.9978 - val_loss: 0.0021 - val_accuracy: 0

In [107]:
pred_acc = model.evaluate(x_test,y_test)
print("Test accuracy is {}".format(pred_acc))

113/113 [==============================] - 1s 7ms/step - loss: 0.0388 - accuracy: 0.9917
Test accuracy is [0.03880178555846214, 0.9916643500328064]


In [108]:
model.save("modelcnnkerastuner.h5")